In [1]:
'''
HGU
CSEE
KGH

Fuzzy art
'''

'\nHGU\nCSEE\nKGH\n\nFuzzy art\n'

In [2]:
from numpy import linalg as LA
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
from numpy import linalg as LA

In [149]:
class Fuzzy_ART():
    def __init__(self, feature_space = 1, threshold = 0.5, w_list = []):
        print("##model is created##")
        self.M = feature_space
        self.threshold = threshold
        self.w_list = w_list
    
    def normalization_vector(self, input_vector):
        pass
        #return normalized_vector
        
    def complement_coding(self, input_vector):
        #normalization_vector(input_vector) 
        complement_vector = [1-i for i in input_vector]
        return complement_vector
    
    def make_input_x(self, input_i):
        return input_i + self.complement_coding(input_i)
       
    def component_wise_min(self, x,y):
        if len(x) != len(y):
            return print("vector length unmatched!")
        zip_list = list(zip(x,y))
        min_list = [min(i,j) for i,j in zip_list]
        return min_list
    
    def area_with_two_vector(self, u , v):
        if len(u) != len(v):
            return print("vector length unmatched!")
        zip_list = list(zip(u,v))
        side_length = [abs(i-j) for i,j in zip_list]
        #cumprod is cumulative multiply all elements = area of two vertex vector
        return np.cumprod(side_length)[-1]     
    
    def choice_function(self, x, w, printable = False):
        '''  
        𝐰_𝑗=(𝐮_𝑗,𝐯 ̅_𝑗 )
        𝐰_𝑗 is an weight vector associated with 𝑗-th category node.
        𝐰_𝑗 represents the area of 𝑗-th category
          '''
        non_zero_a = 0.0001
        #𝐱∧𝐰_𝑗 = (𝐈∧𝐮_𝑗,𝐈 ̅∧𝐯 ̅_𝑗 )=(〖𝐈∧𝐮〗_𝑗, (𝐈∨𝐯_𝑗 ) ̅ )
        #|𝐱∧𝐰_𝑗 |
        x_w = self.component_wise_min(x,w)
        x_w_norm = LA.norm(x_w)
        #|𝐰_𝑗 |
        w_norm = LA.norm(w)
        #𝑇_𝑗 = |𝐱∧𝐰_𝑗 |/(𝛼+|𝐰_𝑗 | )
        T_j = x_w_norm/(non_zero_a + w_norm)
        if printable == True:
                print("𝐱∧𝐰_𝑗:",x_w)
                print("|𝐱∧𝐰_𝑗 |",x_w_norm)
                print("|𝐰_𝑗 |",w_norm)
        if x_w_norm == w_norm: # in the j-category
            return 1.0
        else: #out of j- category
            return T_j
    
    #find nearest w at input x
    def Code_competition(self, input_x, w_list):
        choice_list = [self.choice_function(input_x, w) for w in w_list]
        print("choice list:", choice_list)
        #indices = [i for i, x in enumerate(t_list) if x == 1.0] # is the input in any categories?
        count_in = choice_list.count(1) # num of category where input belongs to
        #if input belong to any w
        if count_in > 0 :
            return self.inner_template_matching(w_list, choice_list)
        #if input not belong to any w
        else:
            return self.outer_template_matching(input_x , w_list, choice_list)
            

    def inner_template_matching(self, w_list, choice_list):
        indices = [i for i, x in enumerate(choice_list) if x == 1.0] # find inner_category which value is 1,0
        # get a |𝐰_𝑗 | to compare 
        inner_category_norm = [LA.norm(w_list[i]) for i in indices]
        # |𝐰_𝑗 |<|𝐰_𝑘 | thus Thus, |𝑅_𝑗 |>|𝑅_𝑘 |
        choice_category = inner_category_norm.index(max(inner_category_norm)) # max |𝐰_𝑗 | was the best choice by ARTMAP policy
        print("inner_template_matching", w_list, indices[choice_category])
        return w_list, indices[choice_category] # the choice of j th category index  𝐽=arg  max┬𝑗⁡〖𝑇_𝑗 〗
        
    def outer_template_matching(self, input_x , w_list ,choice_list):  
        for idx, index in enumerate(np.flip(np.argsort(choice_list))): # sort the up-down and test each in order
            #|𝑅_𝐽⊕𝐈"|=𝑀−|𝐱∧𝐰_𝐽 |≤𝑀(1−𝜌)
            print(w_list[index])
            x_w =self.component_wise_min(input_x , w_list[index])
            matching_bool = ((self.M - LA.norm(x_w))<=(self.M-self.threshold))
            
            if matching_bool == True : # if matching is true, extend that w-th boundary 
                w_list[index] = self.component_wise_min(input_x, w_list[index]) # template learning
                print("outer_template_matching1",  w_list, index )
                return w_list, index
        # if any boundary is unmatched, make new boundary by input_x
        w_list = w_list + [input_x] # Category addition
        print("outer_template_matching2", w_list, len(w_list)-1)
        return  w_list, len(w_list)-1
    
    def ART(self, input_i):
        input_x = self.make_input_x(input_i)
        self.w_list, category  = self.Code_competition(input_x, self.w_list)
        return category
    
    def info(self):
        print("feature space = ". self.M)
        print("threshold = ", self.threshold)
        print("w_list =",self.w_list)
    
    def plot_ART(self):
        pass
        
        
                

            

In [150]:
beta = Fuzzy_ART()

##model is created##


In [151]:
input_1 = [0.1,0.1]

In [152]:
beta.ART(input_1)

choice list: []
outer_template_matching2 [[0.1, 0.1, 0.9, 0.9]] 0


0

In [153]:
input_2 = [0.3,0.3]

In [154]:
beta.ART(input_2)

choice list: [0.7808078385942975]
[0.1, 0.1, 0.9, 0.9]
outer_template_matching1 [[0.1, 0.1, 0.7, 0.7]] 0


0

In [155]:
beta.ART(input_2)

choice list: [1.0]
inner_template_matching [[0.1, 0.1, 0.7, 0.7]] 0


0

In [156]:
input_3 = [0.2,0.2]

In [157]:
beta.ART(input_3)

choice list: [1.0]
inner_template_matching [[0.1, 0.1, 0.7, 0.7]] 0


0

In [158]:
input_4 = [1,1]

In [159]:
beta.ART(input_4)

choice list: [0.14140721551575797]
[0.1, 0.1, 0.7, 0.7]
outer_template_matching2 [[0.1, 0.1, 0.7, 0.7], [1, 1, 0, 0]] 1


1

In [160]:
input_5 = [0.9,0.9]

In [161]:
beta.ART(input_5)

choice list: [0.1999800019998, 0.8999363648893749]
[1, 1, 0, 0]
outer_template_matching1 [[0.1, 0.1, 0.7, 0.7], [0.9, 0.9, 0, 0]] 1


1